# Replica project

### Clustering efforts



In [1]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle
import sys
import sklearn


sys.path.insert(0, "../model/")
from utils import *

sys.path.insert(0, "../web_annotation/")
from utils_clusters import *
from metrics_clusters import * 


data_dir = '/scratch/students/schaerf/'
data_dir = '../data/'

### Making the clusters

In [2]:
type_clustering = input() #'dbscan'

In [3]:
subfolder_dir = input() #'25-05-2022'

In [6]:
if type_clustering == 'dbscan':
    dist = float(input()) #1200
else:
    dist = int(input()) #1200

In [5]:
dist2 = float(input()) #0.12

In [7]:
dist2

0.0

In [8]:
effort = int(input()) #0,1,2

In [9]:
if effort == 0:
    data_file = 'data_sample.csv'
elif effort == 3:
    data_file = 'data.csv'
else:
    data_file = subfolder_dir + '/data_retrain_'+str(effort)+'.csv'

embeds_file = subfolder_dir + '/resnext-101_'+subfolder_dir+'.npy'
map_file = subfolder_dir + '/map2pos.pkl'
cluster_file = subfolder_dir + '/clusters_'+type_clustering+'_'+str(dist)+'_'+subfolder_dir+'_19.pkl'

In [10]:
cluster_file

'07-06-2022/clusters_spectral_clustering_1500_07-06-2022_19.pkl'

In [11]:
clusters = make_clusters_embeddings(data_dir, dist=dist, data_file=data_file, embed_file=embeds_file, type_clustering=type_clustering, dist2=dist2)

(79536, 2)


In [19]:
with open(data_dir + cluster_file, 'wb') as outfile:
    pickle.dump(clusters, outfile)

In [24]:
metadata = pd.read_csv(data_dir + data_file).drop(columns=['level_0'])# 'data_sample.csv')
positives = update_morph(data_dir, '-2022') 

In [25]:
cluster_morph_scores = evaluate_morph(positives, cluster_file, set_splits = ['test', 'val'])
cluster_morph_scores

{'precision': 0.8929078483245151, 'recall': 0.7032323536837082}

### Getting 2d position for visual clustering

In [55]:
map2pos = get_2d_pos(data_dir, embed_file=embeds_file)
with open(data_dir + map_file, 'wb') as outfile:
    pickle.dump(map2pos, outfile)

### Retraining

Annotation based scores

In [26]:
for subfolder_dir in ['25-05-2022', '28-05-2022', '01-06-2022']:
    
    positives = update_morph(data_dir, subfolder_dir)
    novelty_scores = novelty_score(positives, subfolder_dir)
    print(subfolder_dir, novelty_scores)

    morpho_graph_clusters = pd.read_csv(data_dir + 'morphograph_clusters.csv')
    morpho_graph_clusters = morpho_graph_clusters[morpho_graph_clusters['cluster_file'].str.contains(subfolder_dir)]

    scores_clusters = cluster_accuracy(morpho_graph_clusters)
    print(subfolder_dir, scores_clusters)

    

25-05-2022 {'original size': 4966, 'newly added': 224, 'additions to existing clusters': 136, 'number of clusters with new elements': 46, 'new clusters': 36, 'new clusters elements': 88, 'progress': '4.51%'}
25-05-2022 0.41
28-05-2022 {'original size': 4966, 'newly added': 217, 'additions to existing clusters': 163, 'number of clusters with new elements': 53, 'new clusters': 16, 'new clusters elements': 54, 'progress': '4.37%'}
28-05-2022 0.44
01-06-2022 {'original size': 4966, 'newly added': 212, 'additions to existing clusters': 186, 'number of clusters with new elements': 48, 'new clusters': 9, 'new clusters elements': 26, 'progress': '4.27%'}
01-06-2022 0.51


In [28]:
for subfolder_dir in ['07-06-2022']:
    
    positives = update_morph(data_dir, subfolder_dir)
    novelty_scores = novelty_score(positives, subfolder_dir)
    print(subfolder_dir, novelty_scores)

    morpho_graph_clusters = pd.read_csv(data_dir + 'morphograph_clusters.csv')
    morpho_graph_clusters = morpho_graph_clusters[morpho_graph_clusters['cluster_file'].str.contains(subfolder_dir)]

    scores_clusters = cluster_accuracy(morpho_graph_clusters)
    print(subfolder_dir, scores_clusters)


07-06-2022 {'original size': 4966, 'newly added': 42, 'additions to existing clusters': 8, 'number of clusters with new elements': 6, 'new clusters': 8, 'new clusters elements': 34, 'progress': '0.85%'}
07-06-2022 0.99


In [97]:
for subfolder_dir in ['25-05-2022', '28-05-2022', '01-06-2022']:
    embeds_file = subfolder_dir + '/resnext-101_'+subfolder_dir+'.npy'
    data = pd.read_csv(data_dir + data_file)
    embeds = np.load(data_dir + embeds_file, allow_pickle=True) 
    
    print(subfolder_dir, )
    get_scores(embeds, data, positives, reverse_map=True)


25-05-2022


100%|██████████| 13028/13028 [01:08<00:00, 189.20it/s]


all positions 200.63960298654268
min positions 63.51210898082745
median positions 205.69374369323916
mean average precision 0.42130403841280356
recall @ 400 0.77763527381736
recall @ 200 0.7180536846504237
recall @ 100 0.6583113652859789
recall @ 50 0.5929899953228329
recall @ 20 0.5058607889858885
28-05-2022


100%|██████████| 13028/13028 [01:08<00:00, 190.35it/s]


all positions 203.44011266598113
min positions 62.32139253279516
median positions 206.63925327951563
mean average precision 0.4041070277315487
recall @ 400 0.7653663023411815
recall @ 200 0.7272548861374608
recall @ 100 0.6520562339478639
recall @ 50 0.5890477673405604
recall @ 20 0.49320241250773944
01-06-2022


100%|██████████| 13028/13028 [01:02<00:00, 207.57it/s]


all positions 205.89131309518487
min positions 61.12865792129163
median positions 210.72073662966702
mean average precision 0.40791816048701385
recall @ 400 0.7829580650169901
recall @ 200 0.720258031442242
recall @ 100 0.65470614163455
recall @ 50 0.5832405151719773
recall @ 20 0.4948517078250467


In [113]:
cluster_files = ['25-05-2022/clusters_dbscan_0.08_25-05-2022_19.pkl', '25-05-2022/clusters_mix_2000_25-05-2022_19.pkl',
                '28-05-2022/clusters_dbscan_0.08_28-05-2022_19.pkl', '28-05-2022/clusters_mix_2000_28-05-2022_19.pkl',
                '01-06-2022/clusters_dbscan_0.08_01-06-2022_19.pkl', '01-06-2022/clusters_mix_2000_01-06-2022_19.pkl',
                ]
for cluster_file in cluster_files:
    cluster_morph_scores = evaluate_morph(positives, cluster_file , set_splits = ['test', 'val'])
    print(cluster_file, cluster_morph_scores)

25-05-2022/clusters_dbscan_0.08_25-05-2022_19.pkl {'precision': 0.9512017990120183, 'recall': 0.7429868891724661}
25-05-2022/clusters_mix_2000_25-05-2022_19.pkl {'precision': 0.8256996054683917, 'recall': 0.5669260316628028}
28-05-2022/clusters_dbscan_0.08_28-05-2022_19.pkl {'precision': 0.9571173939604596, 'recall': 0.7451539260919626}
28-05-2022/clusters_mix_2000_28-05-2022_19.pkl {'precision': 0.7745777991678072, 'recall': 0.5215796326504856}
01-06-2022/clusters_dbscan_0.08_01-06-2022_19.pkl {'precision': 0.9060931130912883, 'recall': 0.7226228741647304}
01-06-2022/clusters_mix_2000_01-06-2022_19.pkl {'precision': 0.779881047315258, 'recall': 0.5556236558471487}


#### Making the new train data

In [122]:
metadata = pd.read_csv(data_dir + data_file).drop(columns=['level_0'])# 'data_sample.csv')
data = pd.concat([metadata, positives], axis=0).reset_index().groupby(['uid', 'uid_connection']).last().reset_index()
print(data.shape)

(12825, 27)


In [11]:
data.to_csv(data_dir + 'data_retrain_' + str(effort+1) + '.csv', index=False)

In [13]:
metadata = metadata.drop(columns=['img1', 'img2', 'type', 'annotated', 'index', 'cluster', 'set', 'uid_connection'])
embeddings = np.load(data_dir + embeds_file, allow_pickle=True) 


with open(data_dir + 'uid2path.pkl', 'rb') as infile:
    uid2path = pickle.load(infile)

In [ ]:
sets = make_new_train_set(embeddings, positives, morpho_graph_clusters, subfolder_dir, uid2path)

In [16]:
train_set = sets[sets['set'] == 'train']
val_set = sets[sets['set'] == 'val']

train_set.to_csv(data_dir + 'retrain_' + str(effort+1) + '_train.csv')
val_set.sample(frac=0.1).to_csv(data_dir + 'retrain_' + str(effort+1) + '_val.csv')

### Evaluating the re-training

In [134]:
subfolder_dir = '28-05-2022'
cluster_file = '28-05-2022/clusters_mix_2000_28-05-2022_19.pkl'
previous_cluster_date = '25-05-2022'

In [147]:
morpho_graph_clusters = pd.read_csv(data_dir + 'morphograph_clusters.csv')
track_cluster_progression(morpho_graph_clusters, cluster_file, previous_cluster_date, positives)

(3174, 7)
(936, 8)


{'positives': 0.72, 'negatives': 0.6892430278884463}

In [148]:
subfolder_dir = '01-06-2022'
cluster_file = '01-06-2022/clusters_mix_2000_01-06-2022_19.pkl'
previous_cluster_date = '28-05-2022'

morpho_graph_clusters = pd.read_csv(data_dir + 'morphograph_clusters.csv')
track_cluster_progression(morpho_graph_clusters, cluster_file, previous_cluster_date, positives)

(1720, 7)
(840, 8)


{'positives': 0.7441860465116279, 'negatives': 0.7454545454545455}

In [149]:
previous_cluster_date = '25-05-2022'

morpho_graph_clusters = pd.read_csv(data_dir + 'morphograph_clusters.csv')
track_cluster_progression(morpho_graph_clusters, cluster_file, previous_cluster_date, positives)

(3174, 7)
(936, 8)


{'positives': 0.8, 'negatives': 0.6054794520547945}